In [2]:
import pandas as pd
df = pd.read_csv("card_cust.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           1000 non-null   int64  
 1   BALANCE                           1000 non-null   float64
 2   BALANCE_FREQUENCY                 1000 non-null   float64
 3   PURCHASES                         1000 non-null   float64
 4   ONEOFF_PURCHASES                  1000 non-null   float64
 5   INSTALLMENTS_PURCHASES            1000 non-null   float64
 6   CASH_ADVANCE                      1000 non-null   float64
 7   PURCHASES_FREQUENCY               1000 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 11  CASH_ADVANCE_TRX                  1000 non-null   float64
 12  PURCHAS

In [71]:
df.isna().sum()

CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

In [8]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           1000 non-null   int64  
 1   BALANCE                           1000 non-null   float64
 2   BALANCE_FREQUENCY                 1000 non-null   float64
 3   PURCHASES                         1000 non-null   float64
 4   ONEOFF_PURCHASES                  1000 non-null   float64
 5   INSTALLMENTS_PURCHASES            1000 non-null   float64
 6   CASH_ADVANCE                      1000 non-null   float64
 7   PURCHASES_FREQUENCY               1000 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 11  CASH_ADVANCE_TRX                  1000 non-null   float64
 12  PURCHAS

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

# Q1

In [12]:
df1 = df.copy()
df1.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [72]:
# df1[["TENURE","BALANCE","CREDIT_LIMIT"]].corr().reset_index().replace(1,0).max()

In [20]:
round(0.948405, 2)

0.95

In [19]:
df.groupby("TENURE")["BALANCE","CREDIT_LIMIT"].corr().reset_index().replace(1,0).max()

C:\Users\younl\AppData\Local\Temp/ipykernel_8304/1735516273.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby("TENURE")["BALANCE","CREDIT_LIMIT"].corr().reset_index().replace(1,0).max()


TENURE                  12.0
level_1         CREDIT_LIMIT
BALANCE             0.948405
CREDIT_LIMIT        0.948405
dtype: object

In [76]:
df_corr = df.groupby("TENURE")["BALANCE","CREDIT_LIMIT"].corr()
df_corr.xs("BALANCE", level = 1)["CREDIT_LIMIT"].round(2).max()

C:\Users\younl\AppData\Local\Temp/ipykernel_8304/4153301470.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_corr = df.groupby("TENURE")["BALANCE","CREDIT_LIMIT"].corr()


0.95

# Q2

In [32]:
df2 = df.copy()
df22 = df2.copy()

In [26]:
len(df2.columns)

18

In [31]:
df2.columns
df2_indep = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']

In [33]:
zsm = StandardScaler().fit(df2[df2_indep])
df22[df2_indep] = zsm.transform(df2[df2_indep])

In [36]:
kl = [2,3,4,5]
for n in kl:
    model = KMeans(n_clusters=n, random_state=1234).fit(df22[df2_indep])
    acc = silhouette_score(df22[df2_indep] , model.labels_)
    print(n, acc)
    

2 0.3075281530456079
3 0.19636128772937608
4 0.20715098494639902
5 0.19274056144483248


n이 2일때 가장 큰데 이때 제일 좋다.

In [37]:
model = KMeans(n_clusters=2, random_state=1234).fit(df22[df2_indep])

In [41]:
df2["label"] = model.labels_

In [44]:
round(df2.groupby("label")["ONEOFF_PURCHASES"].mean().max(), 2)

3946.19

# Q3

In [45]:
df3 = df.copy()

In [47]:
train = df3.loc[df3["CUST_ID"]% 4 != 0,]
test = df3.loc[df3["CUST_ID"]% 4 == 0,]
len(train), len(test)

(752, 248)

In [49]:
train.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

In [53]:
indep_l = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']

dep_l = "ONEOFF_PURCHASES"

In [54]:
model = DecisionTreeRegressor(random_state=1234).fit(X = train[indep_l], y = train[dep_l])

In [61]:
pred = model.predict(X = test[indep_l])
len(pred)

248

In [56]:
from sklearn.metrics import accuracy_score

In [57]:
accuracy_score(y_true=test[dep_l], y_pred=pred)

ValueError: continuous is not supported

In [81]:
from sklearn.metrics import mean_squared_error
accuracy_score(y_true=test[dep_l], y_pred=pred)

ValueError: continuous is not supported

In [62]:
test["pred"] = pred

C:\Users\younl\AppData\Local\Temp/ipykernel_8304/1330184522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred"] = pred


In [63]:
test.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,pred
3,10004,1666.670542,0.636364,1499.0,1499.0,0.0,205.788017,0.083333,0.083333,0.0,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.0,12.0,1500.0
7,10008,1823.652743,1.000000,436.2,0.0,436.2,0.000000,1.000000,0.000000,1.0,0.000000,0.0,12.0,2300.0,679.065082,532.033990,0.0,12.0,0.0


In [64]:
test2 = test[[ "ONEOFF_PURCHASES","pred"]]
test2

,ONEOFF_PURCHASES,pred
3,1499.00,1500.00
7,0.00,0.00
11,1492.18,1490.00
15,0.00,0.00
19,0.00,0.00
...,...,...
982,1864.11,1540.41
986,0.00,0.00
990,1375.50,1282.85
994,0.00,0.00


In [65]:
test2["calc"] = (test2["ONEOFF_PURCHASES"] - test2["pred"])**2

C:\Users\younl\AppData\Local\Temp/ipykernel_8304/459620968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2["calc"] = (test2["ONEOFF_PURCHASES"] - test2["pred"])**2


In [66]:
ret = test2["calc"].sum() / test2.shape[0]

In [67]:
ret

1079924.1015294355

In [70]:
round(ret **0.5, 1)

1039.2

In [78]:

y_t = test[ "ONEOFF_PURCHASES"]
y_p = pred

In [79]:
((y_t -y_p)**2).mean()**0.5

1039.193967231063

In [80]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_true = y_t, y_pred= y_p)

1079924.1015294355